# 1 문제정의
### 나이(age) - 소득(income) 간의 로지스틱회귀
### 나이가 증가하면할수록 소득이 높을 확률
```, 1개)있음``

# 2 라이브러리가져오기

In [1]:
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('인구조사DataSet/train.csv')
test = pd.read_csv('인구조사DataSet/test.csv')

train

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,3331,34.0,State-gov,177331,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,Black,Male,4386,0,40.0,United-States,>50K
1,19749,58.0,Private,290661,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States,<=50K
2,1157,48.0,Private,125933,Some-college,10,Widowed,Exec-managerial,Unmarried,Black,Female,0,1669,38.0,United-States,<=50K
3,693,58.0,Private,100313,Some-college,10,Married-civ-spouse,Protective-serv,Husband,White,Male,0,1902,40.0,United-States,>50K
4,12522,41.0,Private,195661,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,54.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29299,15999,28.0,Private,47168,10th,6,Never-married,Machine-op-inspct,Own-child,White,Female,0,0,40.0,United-States,<=50K
29300,21604,44.0,Local-gov,231793,Doctorate,16,Married-spouse-absent,Prof-specialty,Unmarried,White,Female,0,0,38.0,United-States,<=50K
29301,26839,41.0,Local-gov,201435,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,Black,Male,0,0,40.0,United-States,<=50K
29302,16681,43.0,Private,137722,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40.0,United-States,<=50K


# 탐색적 데이터 분석(EDA)

In [2]:
# 데이터 샘플
train.head(3)
# 데이터 크기
train.shape, test.shape
# 자료형 (타입)
train.info()
# 수치형 컬럼 통계값 확인(train)
train.describe()
# 수치형 컬럼 통계값 확인(test)
test.describe()
# 범주형 컬럼 통계값 확인 (train)
train.describe(include='O')
# 범주형 컬럼 통계값 확인 (test)
test.describe(include='object')
# 결측치 (train)
train.isnull().sum()
# 결측치 (test)
test.isnull().sum()
# label(target)별 개수 확인
train['income'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
 15  income          29304 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 3.6+ MB


income
<=50K    22263
>50K      7041
Name: count, dtype: int64

In [3]:
# 데이터 전처리

In [4]:
# 결측치가 있는 데이터(행) 전체 삭제 및 확인 dropna() / 기본값 axis=0
print("처리전:", train.shape)
df = train.dropna()
print("처리후:", df.shape)

처리전: (29304, 16)
처리후: (27096, 16)


In [5]:
# 결측치가 있는 특정컬럼 데이터(행)삭제
df = train.dropna(subset=['native.country','workclass'])
df.isnull().sum()


id                 0
age               12
workclass          0
fnlwgt             0
education          0
education.num      0
marital.status     0
occupation         6
relationship       0
race               0
sex                0
capital.gain       0
capital.loss       0
hours.per.week    13
native.country     0
income             0
dtype: int64

In [6]:
# 결측치가 있는 컬럼 삭제 dropna(axis=1)
print("처리전:", train.shape)
df = train.dropna(axis=1)
print("처리후:", df.shape)

처리전: (29304, 16)
처리후: (29304, 11)


In [7]:
# 결측치가 많은 특정 컬럼 삭제 drop
print("처리전:", train.shape)
df = train.drop(['native.country','workclass'], axis=1)
print("처리후:", df.shape)

처리전: (29304, 16)
처리후: (29304, 14)


In [8]:
# 최빈값
m = train['workclass'].mode()[0]
train['workclass'] = train['workclass'].fillna(m)

m = train['native.country'].mode()[0]
train['native.country'] = train['native.country'].fillna(m)

train.isnull().sum()

id                   0
age                 12
workclass            0
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1668
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week      13
native.country       0
income               0
dtype: int64

In [9]:
# 결측값을 새로운 카테고리로 생성 X_train['occupation']
train['occupation'] = train['occupation'].fillna('X')
train.isnull().sum()

id                 0
age               12
workclass          0
fnlwgt             0
education          0
education.num      0
marital.status     0
occupation         0
relationship       0
race               0
sex                0
capital.gain       0
capital.loss       0
hours.per.week    13
native.country     0
income             0
dtype: int64

In [10]:
# test데이터
test['workclass'] = test['workclass'].fillna(train['workclass'].mode()[0])
test['native.country']  = test['native.country'].fillna(train['native.country'].mode()[0])
test['occupation'] = test['occupation'].fillna("X")

In [11]:
# age 컬럼 평균값으로 채우기
value = int(train['age'].mean())
print("평균값:", value)
train['age'] = train['age'].fillna(value)
test['age'] = test['age'].fillna(value)

평균값: 38


In [12]:
# 주당 근무시간 중앙값으로 채우기
value = int(train['hours.per.week'].median())
print("중앙값:", value)
train['hours.per.week'] = train['hours.per.week'].fillna(value)
test['hours.per.week'] = test['hours.per.week'].fillna(value)

중앙값: 40


In [13]:
## 이상치 처리

In [14]:
# age가 1이상인 데이터만 살림
print(train.shape)
train = train[train['age']>0]
print(train.shape)

(29304, 16)
(29301, 16)


In [15]:
# ------------------------------------------------------------
# [문제] 로지스틱 회귀 TEST
# ------------------------------------------------------------
### 나이(age) - 소득(income) 간의 로지스틱회귀
### 나이가 증가하면할수록 소득이 높을 확률